In [1]:
# Imports
import gymnasium as gym

from stable_baselines3 import PPO
from stable_baselines3.common.env_util import make_vec_env

import pickle

# Import
# Math
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
from numpy.random import Generator, PCG64
import random
from scipy.stats import norm
import scipy.optimize as optimize
import time
import math as m

# Ez parallelization
from joblib import Parallel, delayed

# File Handling
from sys import getsizeof
import _pickle as pickle
# Alternatively use JSON which will be human readable
# import json

import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

import torch as th
import pandas as pd
from stable_baselines3.common.noise import NormalActionNoise, OrnsteinUhlenbeckActionNoise
from stable_baselines3.common.monitor import Monitor
import numpy as np
from matplotlib import pyplot as plt
from typing import Any, List, Sequence, Tuple
from env import tradingEng

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))


# Custom imports
from MarketGeneratingFunctions.global_cache import Global_Cache
import MarketGeneratingFunctions.base_from_gen as bg
import MarketGeneratingFunctions.pricing_func as pf
from MarketGeneratingFunctions.path_datatype import Path

In [2]:
# Define environment
file_path = os.path.abspath("../MarketGeneratingFunctions/1corr100pathsHWRunDemo.pkl")
with open(file_path,"rb") as fp:
    paths = pickle.load(fp)
env = tradingEng(paths)

/Users/isabellebyman/opt/anaconda3/envs/exjobb/lib/python3.13/site-packages/gymnasium/spaces/box.py:130: UserWarning: WARN: Box bound precision lowered by casting to float32
  gym.logger.warn(f"Box bound precision lowered by casting to {self.dtype}")


In [3]:
### Unhedged PnL
def pnl_cva(path):
    pnl_without_hedge = np.zeros(len(path.CVA)-1)
    for i in range(1, len(path.CVA)):
        pnl_without_hedge[i-1] = path.CVA[i]-path.CVA[i-1]
    return pnl_without_hedge

pnl_without_hedge = [None]*len(paths)
for i in range(len(pnl_without_hedge)):
    pnl_without_hedge[i] = pnl_cva(paths[i])



### Delta Hedged PnL
if not 'T_s' in vars() or 'T_s' in globals():
    T_s = np.arange(0,11)
from MarketGeneratingFunctions import DeltaHedge
def DeltaFromFunc(path):
    pnl = np.zeros(len(path.CVA)-1)
    for i in range(len(path.CVA)-1):
        t = path.t_s[i]
        Q = [path.Q_s[j][i] for j in range(0,11)]
        Swapts = [path.Swaptions[j][i] for j in range(0,10)]
        T_sl = T_s.copy()
        
        [SwapsHedge,Qhedge] = DeltaHedge.delta_hedge(Swapts,Q,T_sl,t)

        cost = np.inner(SwapsHedge,Swapts) + np.inner(Qhedge,Q)
        
        futureSwapts = [path.Swaptions[j][i+1] for j in range(0,10)]
        futureQ = [path.Q_s[j][i+1] for j in range(0,11)]

        value = np.inner(SwapsHedge,futureSwapts) + np.inner(Qhedge,futureQ)

        pnl[i] = ((path.CVA[i+1] - path.CVA[i]) - (value - cost))
    return pnl

pnl_delta_hedge = [None]*len(paths)
for i in range(len(pnl_without_hedge)):
    pnl_delta_hedge[i] = DeltaFromFunc(paths[i])

ImportError: cannot import name 'DeltaHedge' from 'MarketGeneratingFunctions' (/Users/isabellebyman/Documents/Courses_LTH/Exjobb/cva_risk_management_thesis/MarketGeneratingFunctions/__init__.py)

## RL hedge

In [ ]:
modelG = PPO.load("ppo_trading")

def rlHedge(path, model):
    pnl = np.zeros(len(path.CVA)-1)
    for i in range(round((len(path.CVA)-1)/10)):
        # Get State
        lambdax = path.lambdas[i]
        r = path.r[i]
        t = path.t_s[i]

        Q = [path.Q_s[j][i] for j in range(0,11)]
        Swapts = [path.Swaptions[j][i] for j in range(0,10)]

        futureSwapts = [path.Swaptions[j][i+1] for j in range(0,10)]
        futureQ = [path.Q_s[j][i+1] for j in range(0,11)]

        # Create Hedge
        hedgPos = model.predict(np.asarray([lambdax,r,t]))
        hedgPos = hedgPos[0]
        
        #print(hedgPos)

        # Split into swp and Q
        swpPos = hedgPos[0]
        QPos = hedgPos[1]

        # Test Hedge
        cost = Q[-1]*QPos + Swapts[-1]*swpPos
        #print(cost)
        value = futureQ[-1]*QPos + futureSwapts[-1]*swpPos
        #print(value)
        #print(value - cost)
        #print(path.CVA[i+1] - path.CVA[i]) 
        pnl[i] = ((path.CVA[i+1] - path.CVA[i]) - (value - cost))
    return pnl

pnl_RL_hedge = [None]*len(paths)
for i in range(len(pnl_without_hedge)):
    pnl_RL_hedge[i] = rlHedge(paths[i], modelG)